In [1]:
from prepared_data.get_prepared_data import get_prepared_data
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from prepared_data.prepared_test_results import result_predict_prob_to_dataFrame, not_duplicate_elements_in_dataframes
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
import numpy as np

from prepared_data.train_model import split_features_target_and_map_target

2022-05-06 16:41:03.854676: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-06 16:41:03.854698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
train_org = pd.read_csv('data/train.csv')
train, league_mean_ratting_all, teams_ratting_all = get_prepared_data(train_org,number_of_history_matches=4,map_target=False)

test_org = pd.read_csv('data/test.csv')
test,_,_ = get_prepared_data(test_org, 4,league_mean_ratting_all, teams_ratting_all,map_target=False)


/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


percent of object with nan value and orginals:  42.20, 100.000000
remove object with more that  col 50 % empty...
date to datetime type and remove nan date....
percent of object with nan value and orginals:  0.00, 92.767131
percent of object with nan value and orginals:  39.35, 100.000000
remove object with more that  col 50 % empty...
date to datetime type and remove nan date....
percent of object with nan value and orginals:  0.00, 94.684436


,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,...,home_team_history_target_3_1.0,away_team_history_target_3_-1.0,away_team_history_target_3_0.0,away_team_history_target_3_1.0,home_team_history_target_4_-1.0,home_team_history_target_4_0.0,home_team_history_target_4_1.0,away_team_history_target_4_-1.0,away_team_history_target_4_0.0,away_team_history_target_4_1.0
id,,,,,,,,,,,,,,,,,,,,,
11989266,home,1,77,144,189,69,0.0,1.0,0.0,0.0,...,1,1,0,0,0,0,1,1,0,0
17361043,away,1,456,72,168,77,0.0,0.0,1.0,0.0,...,0,0,0,1,0,0,1,1,0,0
17934238,away,1,74,69,103,310,0.0,0.0,1.0,1.0,...,0,0,0,1,0,0,1,0,0,1
17361047,away,1,98,93,74,71,1.0,0.0,0.0,1.0,...,1,0,0,1,1,0,0,0,0,1
17687749,home,1,72,96,72,120,1.0,1.0,0.0,1.0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17887524,home,1,168,167,88,79,1.0,0.0,0.0,1.0,...,0,0,0,1,1,0,0,0,0,1
17882062,draw,1,547,1344,463,165,1.0,0.0,1.0,1.0,...,1,1,0,0,0,0,1,1,0,0
17933781,away,1,98,69,74,141,0.0,0.0,1.0,0.0,...,0,0,0,1,0,0,1,0,1,0


In [9]:
train.columns
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))


Index(['target', 'is_cup', 'home_team_history_match_date_1',
       'home_team_history_match_date_2', 'home_team_history_match_date_3',
       'home_team_history_match_date_4', 'home_team_history_is_play_home_1',
       'home_team_history_is_play_home_2', 'home_team_history_is_play_home_3',
       'home_team_history_is_play_home_4',
       ...
       'home_team_history_target_3_1.0', 'away_team_history_target_3_-1.0',
       'away_team_history_target_3_0.0', 'away_team_history_target_3_1.0',
       'home_team_history_target_4_-1.0', 'home_team_history_target_4_0.0',
       'home_team_history_target_4_1.0', 'away_team_history_target_4_-1.0',
       'away_team_history_target_4_0.0', 'away_team_history_target_4_1.0'],
      dtype='object', length=104)

In [3]:
def build_and_compile_model(norm,n_neurons, dropout, learning_rate):
    model = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(n_neurons, activation= tf.keras.activations.relu),
    # tf.keras.layers.Dropout(dropout),

    tf.keras.layers.Dense(n_neurons, activation= tf.keras.activations.relu),

    tf.keras.layers.Dense(3)

    # tf.keras.layers.Dense(3,activation= tf.keras.activations.softmax)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # loss=tf.keras.losses.SparseCategoricalCrossentropy(),

              metrics=['accuracy'])

    return model

In [4]:

from prepared_data.train_model import callbacks

n_neurons = 10
dropout = 0.0
learning_rate = 0.001
batch_size = None

train_features, target = split_features_target_and_map_target(train)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

dnn_model = build_and_compile_model(normalizer,n_neurons,dropout,learning_rate)

model_full_name = 'logs/fit/dnn_model_h_4'

early_stop, reduce_lr, tensorboard_callback, checkpoint_callback =   callbacks(model_full_name)

history = dnn_model.fit(
    train_features,
    target,
    verbose=1, epochs=3,
   callbacks=[checkpoint_callback, tensorboard_callback],
)



2022-05-06 16:41:20.981810: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 16:41:20.981834: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 16:41:20.981849: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Zenon): /proc/driver/nvidia/version does not exist
2022-05-06 16:41:20.982033: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/3
3216/3216 [==============================] - 4s 1ms/step - loss: 1.0238 - accuracy: 0.4875
Epoch 2/3
3216/3216 [==============================] - 4s 1ms/step - loss: 1.0065 - accuracy: 0.5006
Epoch 3/3
3216/3216 [==============================] - 3s 1ms/step - loss: 1.0033 - accuracy: 0.5025


In [7]:

probability_model = tf.keras.Sequential([dnn_model,
                                         tf.keras.layers.Softmax()])


test_result = probability_model.predict(test)
test_result = result_predict_prob_to_dataFrame(test_result, test)


array([[0.3694033 , 0.2659013 , 0.36469543],
       [0.24921568, 0.20935705, 0.5414272 ],
       [0.31836146, 0.24757883, 0.43405974],
       ...,
       [0.2964891 , 0.32590556, 0.37760538],
       [0.21672882, 0.27920592, 0.50406533],
       [0.27503297, 0.3136828 , 0.41128418]], dtype=float32)

Training for nan value

In [6]:
train_org = pd.read_csv('data/train.csv')
train,_,_ = get_prepared_data(train_org, number_of_history_matches=1)
test_org = pd.read_csv('data/test.csv')
test_history_1,_,_ = get_prepared_data(test_org, number_of_history_matches=1,league_mean_ratting_all=league_mean_ratting_all,teams_mean_ratting_all= teams_ratting_all)
test_history_1 = not_duplicate_elements_in_dataframes(test_history_1.reset_index(), test)
test_history_1

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


percent of object with nan value and orginals:  42.20, 100.000000
remove history columns [2, 3, 4, 5, 6, 7, 8, 9, 10]
percent of object with nan value and orginals:  26.12, 100.000000
remove object with more that  col 50 % empty...
percent of object with nan value and orginals:  26.12, 100.000000
date to datetime type and remove nan date....
percent of object with nan value and orginals:  24.78, 98.210712
filna coach_id and change to is new coach
percent of object with nan value and orginals:  0.09, 98.210712
fill zero nan data
percent of object with nan value and orginals:  0.00, 98.210712
percent of object with nan value and orginals:  39.35, 100.000000
remove history columns [2, 3, 4, 5, 6, 7, 8, 9, 10]
percent of object with nan value and orginals:  33.03, 100.000000
remove object with more that  col 50 % empty...
percent of object with nan value and orginals:  33.03, 100.000000
date to datetime type and remove nan date....
percent of object with nan value and orginals:  32.21, 98.

,is_cup,home_team_history_match_date_1,home_team_history_is_play_home_1,home_team_history_is_cup_1,home_team_history_goal_1,home_team_history_opponent_goal_1,home_team_history_rating_1,home_team_history_opponent_rating_1,away_team_history_match_date_1,away_team_history_is_play_home_1,...,away_team_history_gol_difference_1,away_team_mean_regeneration_time,league_id_ratting,sum_history_targets,home_team_history_target_1_-1.0,home_team_history_target_1_0.0,home_team_history_target_1_1.0,away_team_history_target_1_-1.0,away_team_history_target_1_0.0,away_team_history_target_1_1.0
id,,,,,,,,,,,,,,,,,,,,,
18395531,1,388,1.0,0.0,2.0,1.0,11.078400,3.724950,74,0.0,...,3.0,74.0,6.801392,2.0,0,0,1,0,0,1
18395534,1,96,0.0,0.0,1.0,2.0,6.489692,4.418742,77,0.0,...,2.0,77.0,6.801392,0.0,1,0,0,0,0,1
18395559,1,101,0.0,0.0,1.0,3.0,6.168855,5.675820,98,1.0,...,2.0,98.0,6.801392,0.0,1,0,0,0,0,1
18395595,1,72,1.0,0.0,1.0,0.0,10.392432,3.526532,95,1.0,...,3.0,95.0,6.801392,2.0,0,0,1,0,0,1
18395597,1,76,1.0,0.0,2.0,2.0,9.860605,3.190473,77,0.0,...,0.0,77.0,6.801392,0.0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18433161,1,74,0.0,1.0,0.0,3.0,5.328017,12.617483,1317,0.0,...,0.0,1317.0,8.857095,-1.0,1,0,0,0,1,0
18430451,1,72,1.0,1.0,1.0,0.0,6.779631,10.878485,69,1.0,...,-4.0,69.0,8.857095,0.0,0,0,1,1,0,0
18433160,1,78,0.0,0.0,1.0,4.0,5.731083,13.314733,5662,1.0,...,-1.0,5662.0,8.857095,-2.0,1,0,0,1,0,0


In [7]:
target = train['target']
X_train_org = train.drop('target', axis=1)
pipe = make_pipeline(MinMaxScaler(),
                     LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000, n_jobs=-1))
pipe.fit(X_train_org, target)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, multi_class='ovr', n_jobs=-1,
                                    random_state=0))])

In [8]:
test_result_history_1 = pipe.predict_proba(test_history_1)
test_result_history_1 = result_predict_prob_to_dataFrame(test_result_history_1, test_history_1)
test_result_history_1

,id,away,draw,home
0,18395531,0.326631,0.217357,0.456012
1,18395534,0.401950,0.226799,0.371251
2,18395559,0.419427,0.221510,0.359063
3,18395595,0.445531,0.223663,0.330806
4,18395597,0.327524,0.222984,0.449493
...,...,...,...,...
9354,18433161,0.466112,0.162647,0.371241
9355,18430451,0.236442,0.135125,0.628432
9356,18433160,0.412746,0.161971,0.425283
9357,18430453,0.476034,0.162803,0.361163


In [9]:
from prepared_data.prepared_test_results import append_test_results

test_result_learn = append_test_results(test_result_history_1, test_result)
test_result_learn

,id,away,draw,home
0,18395531,0.326631,0.217357,0.456012
1,18395534,0.401950,0.226799,0.371251
2,18395559,0.419427,0.221510,0.359063
3,18395595,0.445531,0.223663,0.330806
4,18395597,0.327524,0.222984,0.449493
...,...,...,...,...
68358,18125501,0.225129,0.272082,0.502789
68359,18070396,0.361701,0.333688,0.304611
68360,18094517,0.328333,0.310938,0.360729
68361,18163535,0.313334,0.295313,0.391354


In [10]:
from prepared_data.prepared_test_results import completed_test_result_and_save

result_all = completed_test_result_and_save(test_org, test_result_learn, 'logist_regression_history_8_+_1')
result_all.shape[0]  == test_org.shape[0]


Number of not duplicate elements  4348


True